In [1]:
library(EWCE) # Versions prior to 1.5.5 did not compute controlled results properly
library(orthogene)
library(Seurat)

Loading required package: RNOmni

Registered S3 methods overwritten by 'treeio':
  method              from    
  MRCA.phylo          tidytree
  MRCA.treedata       tidytree
  Nnode.treedata      tidytree
  Ntip.treedata       tidytree
  ancestor.phylo      tidytree
  ancestor.treedata   tidytree
  child.phylo         tidytree
  child.treedata      tidytree
  full_join.phylo     tidytree
  full_join.treedata  tidytree
  groupClade.phylo    tidytree
  groupClade.treedata tidytree
  groupOTU.phylo      tidytree
  groupOTU.treedata   tidytree
  inner_join.phylo    tidytree
  inner_join.treedata tidytree
  is.rooted.treedata  tidytree
  nodeid.phylo        tidytree
  nodeid.treedata     tidytree
  nodelab.phylo       tidytree
  nodelab.treedata    tidytree
  offspring.phylo     tidytree
  offspring.treedata  tidytree
  parent.phylo        tidytree
  parent.treedata     tidytree
  root.treedata       tidytree
  rootnode.phylo      tidytree
  sibling.phylo       tidytree



### setup

In [11]:
wd <- '~/codebases/MacBrainDev/'
data.dir <- 'data/'
setwd(wd)


n.cores <-  as.numeric(Sys.getenv('SLURM_CPUS_PER_TASK'))
n.cores <- if (!is.na(n.cores) & n.cores > 1) n.cores else parallel::detectCores()
n.cores

options(future.globals.maxSize= 5*1024^3)
future::plan(strategy = "multicore", workers = n.cores)

disease_lists <- readRDS(glue::glue('{data.dir}Disease_genes/all_diseases_list.rds'))

# For markers only
downsample_ncells_per_subtype <- Inf

# Define filenames

base.name <- 'All.MNN.v1.org.fct'

indata.fname <- paste0(data.dir, base.name, '.rds')
# ctd object
ctd.fname <- paste0(data.dir, 'ewce_ctd.', base.name,
                    '.ds', downsample_ncells_per_subtype, '.rds')
# markers
markers.fname <- paste0(data.dir, 'ewce_downsampled_markers.',
                        base.name,  '.ds', 
                        downsample_ncells_per_subtype,'.rds')
# sct object
sct.fname <- paste0(data.dir, 'ewce_sct.', base.name, '.rds')
# sct normalized object
sct.normed.fname <- paste0(data.dir, 'ewce_sct_normed_markers_orthos.', base.name, 
                           '.ds', downsample_ncells_per_subtype, '.rds')

# Define parameters
# for ewce
reps <- 20000 #  >10000
verbose <- T

# ewce
dis.res.folder <- paste0(data.dir, 'EWCE_diseases.ds', downsample_ncells_per_subtype, '/')
dir.create(dis.res.folder, showWarnings = F)
EWCE_results.fname <- paste0('EWCE_results.', reps, '_reps.ds', downsample_ncells_per_subtype, '.csv')


[1] 96

In [6]:
# Load data
indata <- readRDS(indata.fname)
# Print summary
indata

Idents(indata) <- 'subtype'

Loading required package: SeuratObject

Loading required package: sp

The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)

Loading required package: Seurat



An object of class Seurat 
34619 features across 761529 samples within 1 assay 
Active assay: RNA (34619 features, 0 variable features)
 2 dimensional reductions calculated: mnn, umap

### Find All Marker genes

In [7]:
normalizePath(markers.fname)

[1] "/users/genomics/xoel/codebases/MacBrainDev/data/ewce_downsampled_markers.All.MNN.v1.org.fct.dsInf.rds"

In [9]:
markers.fname

# Check if run
if (!file.exists(markers.fname) & !file.exists(ctd.fname)){
    
    print('Computing Markers')
    
    Idents(indata) <- 'subtype'
    
    all.markers <- FindAllMarkers(
      # ds.seurat,
      indata,
      assay = 'RNA',
      features = NULL,
      logfc.threshold = 0.25,
      test.use = "wilcox",
      slot = "data",
      min.pct = 0.1,
      min.diff.pct = -Inf,
      node = NULL,
      verbose = TRUE,
      only.pos = FALSE,
      max.cells.per.ident = downsample_ncells_per_subtype,
      random.seed = 1,
      latent.vars = NULL,
      min.cells.feature = 3,
      min.cells.group = 3,
      pseudocount.use = 1,
      mean.fxn = NULL,
      fc.name = NULL,
      base = 2,
      return.thresh = 0.01,
      densify = FALSE,
    )
    
    saveRDS(all.markers, markers.fname)
    
} else {
    all.markers <- readRDS(markers.fname)
}

# ADJUSTED P-VALUE < 0.05
length(unique(dplyr::filter(all.markers, p_val_adj < 0.05)$gene))

# ADJUSTED P-VALUE < 0.05 & |AVG log2 FC| > 1 
length(unique(dplyr::filter(all.markers, p_val_adj < 0.05, abs(avg_log2FC) > 1)$gene)) 

# ADJUSTED P-VALUE < 0.05 & |AVG log2 FC| > 2
length(unique(dplyr::filter(all.markers, p_val_adj < 0.05, abs(avg_log2FC) > 2)$gene)) 

significant.markers <- unique(dplyr::filter(all.markers, p_val_adj < 0.05)$gene)

[1] "data/ewce_downsampled_markers.All.MNN.v1.org.fct.dsInf.rds"

[1] 9146

[1] 2503

[1] 519

### SCT Normalization

In [12]:
# Lower number of cores to reduce memory usagefuture::plan(strategy = "multicore", workers = min(40, n.cores))

file.exists(sct.normed.fname)
ctd.fname
file.exists(ctd.fname)
# If CTD is already computed, we don't need anything here
if (file.exists(sct.normed.fname) & !file.exists(ctd.fname)){
    exp.normed <- readRDS(sct.normed.fname)
    
} else if (!file.exists(ctd.fname)){
    
    # If we already have the sct-data, we can load it, otherwise we run it
    if (file.exists(sct.fname)){
        sct <- readRDS(sct.fname)
    } else {
        requireNamespace("sctransform")

        raw.expression <- indata[['RNA']]@counts
        # First, perform sctransform of data
        sct <- sctransform::vst(umi = raw.expression,
                                method="glmGamPoi", 
                                return_cell_attr = TRUE, 
                                n_genes = 3000,
                                verbosity = 2)
        saveRDS(sct, sct.fname)
    }
    
    ## Get orthologues data
    # Check number of genes in SCT-data and in significant markers
    print(paste('Number of genes before SCT:', nrow(indata)))
    sct.genes <- rownames(sct$y)
    print(paste('Number of genes after SCT:', length(sct.genes)))
    print(paste('Number of marker genes:', length(significant.markers)))
    # Intersect lists
    markers.sct.genes <- sct.genes[sct.genes %in% significant.markers]
    print(paste('Number of genes after SCT in markers:', length(markers.sct.genes)))
    # Convert macaque to human genes
    orthos <- convert_orthologs(markers.sct.genes,input_species = 'macaque', verbose = F)
    print(paste('Number of matching orthologs:', nrow(orthos)))

    ## Correct orthologues data
    # Get subset of data
    ortho.raw <- indata[['RNA']]@counts[orthos$input_gene,]

    # Correct subset data using full model
    ortho.sct <- sctransform::correct_counts(x = sct, umi = ortho.raw, verbosity = 2)
    # Rename genes in object
    rownames(ortho.sct) <- rownames(orthos)
    # Normalize gene expression
    exp.normed <- Matrix::t(Matrix::t(ortho.sct) * (1/Matrix::colSums(ortho.sct)))
    
    saveRDS(exp.normed, sct.normed.fname)
    
}

[1] TRUE

[1] "data/ewce_ctd.All.MNN.v1.org.fct.dsInf.rds"

[1] TRUE

### CellTypeData object

In [13]:
# increase number of cores for parallel computing
future::plan(strategy = "multicore", workers = min(80, n.cores))

# Define annotation levels
annot.levels <- as.list(indata@meta.data[,c('subclass','subtype')])

if (file.exists(ctd.fname)){

    ctd <- ctd.fname

} else {

    ctd <- generate_celltype_data(
    
    exp=exp.normed,
    annotLevels=annot.levels,
    groupName='MacaqueDev',
    
    input_species='hsapiens',
    output_species='hsapiens',
    convert_orths=F,
    
    specificity_quantiles=T,
    dendrograms=T,
    
    file_prefix=paste0('ewce_ctd.',base.name),
    force_new_file=T,
    
    no_cores=n.cores,
    as_sparse = TRUE,
    as_DelayedArray = FALSE,
    verbose=T)
    
    file.copy(ctd, ctd.fname)
}

### Perform analysis

In [14]:
print(Sys.time())

[1] "2023-10-13 15:46:21 CEST"


In [15]:
sapply(disease_lists, length)

if (is.character(ctd)){load(ctd)}

# Start buffer of results
dis_results <- list()

max_tries <- 10
geneSizeControl <- F

# Iterate lists of diseases
for (dis in names(disease_lists)){
    
    # get genset
    gset <- disease_lists[[dis]]

    # Decide which level of annotation to analyse, we do both
    for (level in c(1:2)){

        # Start summary
print(paste(which(names(disease_lists)==dis), length(disease_lists), sep='/'))
        print(paste('### Disease:', dis))
        print(paste('### GC&Length control:', geneSizeControl))
        print(paste('### Level:', level))
        
        # define a name of the run and filename
        tag <- paste(dis, level, geneSizeControl) 
        dis_results.fname <- paste0(dis.res.folder, 'EWCE_results.', reps, '_reps.', tag, '.csv')

        # If results were computed, load them
        if (file.exists(dis_results.fname)){
            print('Loading results')
            dis_results[[tag]] <- read.csv(dis_results.fname, row.names=1)
        # If not, try to compute it upto max.tries
        } else {
            tries <- 0
            while (!tag %in% names(dis_results) & tries < max_tries){
                tryCatch(
                    {
                        tries <- tries +1
                        res <- bootstrap_enrichment_test(

                            sct_data = ctd,

                            hits = gset,
                            bg = NULL,

                            genelistSpecies = 'human',
                            sctSpecies = 'human',
                            output_species = "human",

                            reps = reps,

                            annotLevel = level,
                            geneSizeControl = geneSizeControl,
                            controlledCT = NULL,
                            mtc_method = "BH",

                            no_cores = n.cores,
                            sort_results = TRUE,
                            verbose = verbose)


                        res$results$hit.cells <- res$hit.cells[rownames(res$results)]
                        res$results$geneSizeControl <- geneSizeControl
                        
                        # Save, export and delete
                        dis_results[[tag]] <- data.frame(res$results, list=dis)
                        write.csv(dis_results[[tag]], dis_results.fname)
                        rm(res)
                    },
                    error=function(cond) {
                        print(as.character(cond))

                        print(tag)
                        if (tries > max_tries){
                            print('COULD NOT COMPUTE')
                        } else {
                            print('Repeating')
                        }
                        # Choose a return value in case of error
                        return()
                    })
                gc()
            }
        }
        print('next...')
    }
}

# Combine results
results <- do.call('rbind', dis_results)
rownames(results) <- 1:nrow(results)
results

# Export results
write.csv(results, file=EWCE_results.fname)

ASD HC65 
                                      65 
                                      DD 
                                      93 
                            SFARI Score1 
                                     207 
                            SFARI Score2 
                                     211 
                            SFARI Score3 
                                     506 
                         SFARI Syndromic 
                                     126 
                         GWAS::ADHD 2019 
                                      38 
                           GWAS::AD 2019 
                                     192 
                           GWAS::AN 2019 
                                     107 
                          GWAS::ASD 2019 
                                      29 
                           GWAS::BD 2019 
                                     218 
                           GWAS::IQ 2018 
                                     859 
                          GWAS::MDD 2018 
                                      69 
                       GWAS::NEUROT 2018 
                                     300 
                           GWAS::PD 2014 
                                      34 
                          GWAS::SCZ 2020 
                                     130 
                        MAGMA::ADHD 2019 
                                     200 
                          MAGMA::AD 2019 
                                     200 
                          MAGMA::AN 2019 
                                     200 
                         MAGMA::ASD 2019 
                                     200 
                          MAGMA::BD 2019 
                                     200 
                          MAGMA::IQ 2018 
                                     200 
                         MAGMA::MDD 2018 
                                     200 
                      MAGMA::NEUROT 2018 
                                     200 
                         MAGMA::OCD 2018 
                                     200 
                          MAGMA::PD 2014 
                                     200 
                         MAGMA::SCZ 2020 
                                     200 
                          MAGMA::TS 2019 
                                     200 
       DISGENET::Giant Cell Glioblastoma 
                                      84 
                  DISGENET::Glioblastoma 
                                      79 
       DISGENET::Glioblastoma Multiforme 
                                     111 
                        DISGENET::Glioma 
                                      87 
DISGENET::M.Oligoastr+Oligodendrogliomas 
                                      32 
              DISGENET::Malignant Glioma 
                                      70 
              DISGENET::Medulloblastomas 
                                      50 
                 DISGENET::Neuroblastoma 
                                      47

[1] "1/36"
[1] "### Disease: ASD HC65"
[1] "### GC&Length control: FALSE"
[1] "### Level: 1"
[1] "Loading results"
[1] "next..."
[1] "1/36"
[1] "### Disease: ASD HC65"
[1] "### GC&Length control: FALSE"
[1] "### Level: 2"
[1] "Loading results"
[1] "next..."
[1] "2/36"
[1] "### Disease: DD"
[1] "### GC&Length control: FALSE"
[1] "### Level: 1"
[1] "Loading results"
[1] "next..."
[1] "2/36"
[1] "### Disease: DD"
[1] "### GC&Length control: FALSE"
[1] "### Level: 2"
[1] "Loading results"
[1] "next..."
[1] "3/36"
[1] "### Disease: SFARI Score1"
[1] "### GC&Length control: FALSE"
[1] "### Level: 1"
[1] "Loading results"
[1] "next..."
[1] "3/36"
[1] "### Disease: SFARI Score1"
[1] "### GC&Length control: FALSE"
[1] "### Level: 2"
[1] "Loading results"
[1] "next..."
[1] "4/36"
[1] "### Disease: SFARI Score2"
[1] "### GC&Length control: FALSE"
[1] "### Level: 1"
[1] "Loading results"
[1] "next..."
[1] "4/36"
[1] "### Disease: SFARI Score2"
[1] "### GC&Length control: FALSE"
[1] "### Level: 2"


,CellType,annotLevel,p,fold_change,sd_from_mean,q,hit.cells,geneSizeControl,list
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>
1,Excitatory_neurons,1,0.00005,1.6210245,4.949783,0.00075000,5.1582406,FALSE,ASD HC65
2,Inhibitory_neurons,1,0.00025,1.6011505,4.651712,0.00187500,4.7708344,FALSE,ASD HC65
3,gIPC,1,0.05595,1.1455764,1.676427,0.27975000,3.7065873,FALSE,ASD HC65
4,OPC&Oligo,1,0.09410,1.1895992,1.392667,0.28470000,4.5192011,FALSE,ASD HC65
5,CR,1,0.09490,1.1910740,1.366746,0.28470000,3.8939663,FALSE,ASD HC65
6,enIPC,1,0.11695,1.1109784,1.144413,0.29237500,2.7927076,FALSE,ASD HC65
7,Astro,1,0.30115,1.0635441,0.460351,0.64532143,3.9141276,FALSE,ASD HC65
8,inIPC,1,0.87095,0.8844217,-1.075973,0.99990000,2.4041449,FALSE,ASD HC65
9,RB&Vas,1,0.91495,0.7682445,-1.260677,0.99990000,2.9470457,FALSE,ASD HC65


In [17]:
print(Sys.time())

[1] "2023-10-13 15:46:35 CEST"


In [ ]:
sessionInfo()

R version 4.3.1 (2023-06-16)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.5 LTS

Matrix products: default
BLAS/LAPACK: /users/genomics/xoel/micromamba/envs/Portfolio/lib/libopenblasp-r0.3.23.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Europe/Madrid
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] Seurat_4.3.0.1     SeuratObject_4.1.3 sp_2.0-0           EWCE_1.8.2        
[5] RNOmni_1.0.1.2    

loaded via a namespace (and not attached):
  [1] RcppAnnoy_0.0.21              splines_4.3.1                
  [3] later_1.3.1                   pbdZMQ_0.3-9          